### Install lib 

In [3]:
%pip install meteostat pandas xlrd datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [meteostat]/4 [datetime]
Note: you may need to restart the kernel to use updated packages.


#### Data meteostat: Historical weather data of 8 departments in ile de france 


In [4]:
from meteostat import Point, Daily,Hourly
from datetime import datetime
import pandas as pd

## Meteostat Data structure <br>
#### time : The datetime of the observation <br>
 temp : The air temperature in celsius <br>
 dwpt : the dew point in Celsius <br>
 rhum : The relative humidity in percent (%) <br>
 prcp : The one hour precipitation total in mm <br>
 snow : The snow depth in mm <br>
 wdir : The average wind direction in degrees (°) <br>
 wspd : The average wind speed in km/h <br>
 wpgt : The peak wind gust in km/h <br>
 pres : The average sea-level air pressure in hPa <br>
 tsun : The one hour sunshine total in minutes (m) <br>
 coco : The [weather condition code ](https://dev.meteostat.net/formats.html#weather-condition-codes)

In [5]:
# Define 8 coordinates
idf_points = {
    "Paris": (48.8566, 2.3522),
    "Hauts-de-Seine": (48.8924, 2.2060), # Nanterre
    "Seine-Saint-Denis": (48.9109, 2.4395), # Bobigny
    "Val-de-Marne": (48.7904, 2.4556), #Créteil
    "Yvelines": (48.8049, 2.1204), # Versailles
    "Essonne": (48.6290, 2.4410), # Evry
    "Val-d'Oise ": (49.0360, 2.0630), # Cergy
    "Seine-et-Marne": (48.5390, 2.6550) #Melun
}

# Define time range
start = datetime(2024, 1, 1)
end = datetime(2024, 12, 31)

# Retrieve daily data for each city
dfs = []
for city, (lat, lon) in idf_points.items():
    p = Point(lat, lon)
    data = Daily(p, start, end).fetch().reset_index()
    data['city'] = city
    dfs.append(data)

weather_all = pd.concat(dfs, ignore_index=True)
weather_all



,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,city
0,2024-01-01,8.2,6.2,10.9,4.1,<NA>,<NA>,16.4,46.0,1009.3,93.0,Paris
1,2024-01-02,10.9,9.9,11.8,12.2,<NA>,<NA>,28.0,83.0,998.6,0.0,Paris
2,2024-01-03,10.8,9.1,12.7,6.2,<NA>,<NA>,20.2,65.0,999.8,108.0,Paris
3,2024-01-04,9.6,7.6,11.1,0.0,<NA>,<NA>,16.4,46.0,1006.2,137.0,Paris
4,2024-01-05,7.9,6.1,9.9,0.2,<NA>,<NA>,12.2,48.0,1004.3,253.0,Paris
...,...,...,...,...,...,...,...,...,...,...,...,...
2923,2024-12-27,1.3,-1.3,4.9,0.2,<NA>,<NA>,5.0,18.0,1032.9,319.0,Seine-et-Marne
2924,2024-12-28,-0.7,-1.8,0.8,0.2,<NA>,<NA>,8.2,35.0,1031.4,40.0,Seine-et-Marne
2925,2024-12-29,1.9,0.4,3.3,0.0,<NA>,<NA>,7.4,21.0,1032.3,0.0,Seine-et-Marne
2926,2024-12-30,2.2,0.8,3.3,0.0,<NA>,<NA>,8.1,22.0,1032.6,22.0,Seine-et-Marne


In [6]:
weather_all.isnull().sum()

time       0
tavg       0
tmin       0
tmax       0
prcp      12
snow    2893
wdir    2928
wspd       0
wpgt     366
pres       0
tsun     366
city       0
dtype: int64

In [7]:
weather_all.describe()

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
count,2928,2928.0,2928.0,2928.0,2916.0,35.0,0.0,2928.0,2562.0,2928.0,2562.0
mean,2024-07-01 12:00:00.000000256,12.602459,9.084665,16.380089,2.305178,1.0,<NA>,12.472643,38.385051,1016.244433,190.552693
min,2024-01-01 00:00:00,-3.1,-6.9,-2.0,0.0,1.0,<NA>,3.9,13.0,985.0,0.0
25%,2024-04-01 00:00:00,8.5,5.6,11.2,0.0,1.0,<NA>,8.5,28.0,1010.7,36.0
50%,2024-07-01 12:00:00,12.5,9.2,16.3,0.2,1.0,<NA>,11.4,35.0,1016.4,150.0
75%,2024-10-01 00:00:00,16.9,13.0,21.6,2.4,1.0,<NA>,15.2,46.0,1022.6,299.0
max,2024-12-31 00:00:00,28.8,22.0,37.0,73.6,1.0,<NA>,39.7,91.0,1036.6,735.0
std,NaN,6.017009,5.384032,7.044272,5.474174,0.0,<NA>,5.232901,14.012413,9.844798,172.85244


#### Drop snow, tsun


In [8]:
feature_meteo = ["time","temp","dwpt","rhum","prcp","wdir","wspd","pres","coco","city"]


In [9]:
# df_weather = weather_all[feature_meteo]

#### Data electricity ile de france

In [10]:
ele = pd.read_csv( "ile_de_france_ele.xls",sep= "\t",encoding="latin-1")  
ele.head()

,Périmètre,Nature,Date,Heures,Consommation,Thermique,Nucléaire,Eolien,Solaire,Hydraulique,...,TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Unnamed: 29
0,Ile-de-France,Données consolidées,2024-01-01,00:00,7843.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.61,86.9,0.0,0.0,0.01,5.26,1.81,43.96,NaN
1,Ile-de-France,Données consolidées,2024-01-01,00:15,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ile-de-France,Données consolidées,2024-01-01,00:30,8043.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.57,86.9,0.0,0.0,0.01,5.26,1.82,45.20,NaN
3,Ile-de-France,Données consolidées,2024-01-01,00:45,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ile-de-France,Données consolidées,2024-01-01,01:00,7731.0,197.0,0.0,126.0,0.0,1.0,...,ND,1.63,86.9,0.0,0.0,0.01,5.26,1.86,44.58,NaN


In [11]:
ele.columns

Index(['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation', 'Thermique',
       'Nucléaire', 'Eolien', 'Solaire', 'Hydraulique', 'Pompage',
       'Bioénergies', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'Ech. physiques',
       'TCO Thermique (%)', 'TCH Thermique (%)', 'TCO Nucléaire (%)',
       'TCH Nucléaire (%)', 'TCO Eolien (%)', 'TCH Eolien (%)',
       'TCO Solaire (%)', 'TCH Solaire (%)', 'TCO Hydraulique (%)',
       'TCH Hydraulique (%)', 'TCO Bioénergies (%)', 'TCH Bioénergies (%)',
       'Unnamed: 29'],
      dtype='object')

In [12]:
# drop unamed: 29
feature_ele = ['Périmètre', 'Nature', 'Date', 'Heures', 'Consommation', 'Thermique',
       'Nucléaire', 'Eolien', 'Solaire', 'Hydraulique', 'Pompage',
       'Bioénergies', 'Stockage batterie', 'Déstockage batterie',
       'Eolien terrestre', 'Eolien offshore', 'Ech. physiques']
df_ele = ele[feature_ele]

# TCO Thermique (%)', 'TCH Thermique (%)', 'TCO Nucléaire (%)'
#        , 'TCO Eolien (%)', 'TCH Eolien (%)',
#        'TCO Solaire (%)', 'TCH Solaire (%)', 'TCO Hydraulique (%)',
#        'TCH Hydraulique (%)', 'TCO Bioénergies (%)', 'TCH Bioénergies (%)'

In [13]:
df = df_ele.dropna()

In [14]:
df_ele.shape

(35137, 17)

In [15]:
ele.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35137 entries, 0 to 35136
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Périmètre            35137 non-null  object 
 1   Nature               35136 non-null  object 
 2   Date                 35136 non-null  object 
 3   Heures               35136 non-null  object 
 4   Consommation         17568 non-null  float64
 5   Thermique            17568 non-null  float64
 6   Nucléaire            17568 non-null  float64
 7   Eolien               17568 non-null  float64
 8   Solaire              17568 non-null  float64
 9   Hydraulique          17568 non-null  float64
 10  Pompage              17568 non-null  float64
 11  Bioénergies          17568 non-null  float64
 12  Stockage batterie    17568 non-null  float64
 13  Déstockage batterie  17568 non-null  float64
 14  Eolien terrestre     17568 non-null  float64
 15  Eolien offshore      17568 non-null 

In [16]:
# Rename colunms ele
ele = df.rename(columns = {"Périmètre":"Perimeter",
                            "Nucléaire":"Nuclear",
                            "Bioénergies":"Bioenergies",
                            "Déstockage batterie ":"Battery clearance",
                            "TCO Nucléaire (%)":"TCO Nuclear (%)",
                            "TCH Nucléaire (%)":"TCH Nuclear",
                            "TCO Bioénergies (%)":"TCO Bioenergies (%)",
                            "TCH Bioénergies(%)":"TCh Bioenergies (%)"})

In [17]:
ele

,Perimeter,Nature,Date,Heures,Consommation,Thermique,Nuclear,Eolien,Solaire,Hydraulique,Pompage,Bioenergies,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,Ech. physiques
0,Ile-de-France,Données consolidées,2024-01-01,00:00,7843.0,197.0,0.0,126.0,0.0,1.0,0.0,142.0,0.0,0.0,0.0,0.0,7379.0
2,Ile-de-France,Données consolidées,2024-01-01,00:30,8043.0,197.0,0.0,126.0,0.0,1.0,0.0,146.0,0.0,0.0,0.0,0.0,7573.0
4,Ile-de-France,Données consolidées,2024-01-01,01:00,7731.0,197.0,0.0,126.0,0.0,1.0,0.0,144.0,0.0,0.0,0.0,0.0,7264.0
6,Ile-de-France,Données consolidées,2024-01-01,01:30,7559.0,198.0,0.0,127.0,0.0,1.0,0.0,140.0,0.0,0.0,0.0,0.0,7093.0
8,Ile-de-France,Données consolidées,2024-01-01,02:00,7415.0,199.0,0.0,124.0,0.0,1.0,0.0,146.0,0.0,0.0,0.0,0.0,6946.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35126,Ile-de-France,Données consolidées,2024-12-31,21:30,9699.0,149.0,0.0,44.0,0.0,2.0,0.0,140.0,0.0,0.0,0.0,0.0,9363.0
35128,Ile-de-France,Données consolidées,2024-12-31,22:00,9480.0,150.0,0.0,54.0,0.0,2.0,0.0,139.0,0.0,0.0,0.0,0.0,9136.0
35130,Ile-de-France,Données consolidées,2024-12-31,22:30,9599.0,150.0,0.0,67.0,0.0,2.0,0.0,141.0,0.0,0.0,0.0,0.0,9237.0
35132,Ile-de-France,Données consolidées,2024-12-31,23:00,9727.0,148.0,0.0,68.0,0.0,2.0,0.0,142.0,0.0,0.0,0.0,0.0,9364.0


In [18]:
ele.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17568 entries, 0 to 35134
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Perimeter            17568 non-null  object 
 1   Nature               17568 non-null  object 
 2   Date                 17568 non-null  object 
 3   Heures               17568 non-null  object 
 4   Consommation         17568 non-null  float64
 5   Thermique            17568 non-null  float64
 6   Nuclear              17568 non-null  float64
 7   Eolien               17568 non-null  float64
 8   Solaire              17568 non-null  float64
 9   Hydraulique          17568 non-null  float64
 10  Pompage              17568 non-null  float64
 11  Bioenergies          17568 non-null  float64
 12  Stockage batterie    17568 non-null  float64
 13  Déstockage batterie  17568 non-null  float64
 14  Eolien terrestre     17568 non-null  float64
 15  Eolien offshore      17568 non-null  floa

In [19]:
ele['Datetime'] = pd.to_datetime(ele['Date'].astype(str) + ' ' + ele['Heures'].astype(str), errors='coerce')
ele = ele.dropna(subset=['Datetime']).sort_values('Datetime')
ele = ele.set_index('Datetime')
ele_hourly = (ele.resample('24h').agg({
                                    'Consommation': 'mean',
                                    'Thermique': 'mean',
                                    'Nuclear': 'mean',
                                    'Eolien': 'mean',
                                    'Solaire': 'mean',
                                    'Hydraulique': 'mean',
                                    'Pompage': 'mean',
                                    'Bioenergies': 'mean',
                                    'Stockage batterie': 'mean',
                                    'Déstockage batterie': 'mean',
                                    'Eolien terrestre': 'mean',
                                    'Eolien offshore': 'mean',
                                    'Ech. physiques': 'mean'
                                    })
                                    .reset_index()
)

ele_hourly.rename(columns={'Datetime': 'time'}, inplace=True)
ele_hourly

,time,Consommation,Thermique,Nuclear,Eolien,Solaire,Hydraulique,Pompage,Bioenergies,Stockage batterie,Déstockage batterie,Eolien terrestre,Eolien offshore,Ech. physiques
0,2024-01-01,7795.895833,205.770833,0.0,97.500000,16.083333,1.000000,0.0,134.041667,0.0,0.0,0.0,0.0,7342.062500
1,2024-01-02,8409.770833,325.541667,0.0,132.562500,1.520833,1.937500,0.0,137.666667,0.0,0.0,0.0,0.0,7811.250000
2,2024-01-03,8356.979167,342.979167,0.0,131.708333,7.937500,3.000000,0.0,144.270833,0.0,0.0,0.0,0.0,7727.562500
3,2024-01-04,8576.000000,356.354167,0.0,103.625000,6.937500,2.541667,0.0,134.000000,0.0,0.0,0.0,0.0,7973.854167
4,2024-01-05,8777.437500,365.687500,0.0,44.583333,18.479167,2.000000,0.0,143.937500,0.0,0.0,0.0,0.0,8204.145833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2024-12-27,9356.916667,170.416667,0.0,2.916667,25.187500,1.000000,0.0,160.854167,0.0,0.0,0.0,0.0,8997.041667
362,2024-12-28,9681.750000,303.937500,0.0,1.354167,4.166667,1.000000,0.0,145.416667,0.0,0.0,0.0,0.0,9226.416667
363,2024-12-29,9399.937500,161.270833,0.0,0.625000,5.666667,1.291667,0.0,143.854167,0.0,0.0,0.0,0.0,9087.937500
364,2024-12-30,9946.583333,223.041667,0.0,1.250000,2.458333,1.770833,0.0,140.895833,0.0,0.0,0.0,0.0,9577.770833


#### source population : https://idf.handidonnees.fr/page/population

In [20]:
population_idf = {
    "Paris": 2048472,
    "Hauts-de-Seine": 1663974,
    "Seine-Saint-Denis": 1711876,
    "Val-de-Marne": 1426433,
    "Yvelines": 1489363,
    "Essonne": 1343527,
    "Val-d'Oise": 1287589,
    "Seine-et-Marne": 1479615
}
for ele,items in population_idf.items():
    print(ele,items)

Paris 2048472
Hauts-de-Seine 1663974
Seine-Saint-Denis 1711876
Val-de-Marne 1426433
Yvelines 1489363
Essonne 1343527
Val-d'Oise 1287589
Seine-et-Marne 1479615


#### In the reason of disable of data electricity in each department of ile de france, so we need to apply a method that we can obtain from an approximate of utilization of each department base on the each number population of the level aggregation in the IDF. Hence we have the method as follow: 
#### Let: <br>
&nbsp; $E_{d,t}$ &nbsp; is an utilization of the electricity in department d at the ti me t. <br>
&nbsp;   $E_{IDF,t}$ &nbsp; is an utilization of the electricity in IDF at the time t.  <br>
&nbsp;  $Pop_d$ &nbsp; is number of population in departement d. <br>

$$E_{d,t} = E_{IDF,t} \times \frac{Pop_d}{\sum_d{Pop_d}}$$

In [21]:
total_pop = sum(population_idf.values())

In [22]:
# create new columns cosommation electricity base on population in each department 
ele_dept = (
    pd.DataFrame({
        'time': pd.Series(ele_hourly['time'].values.repeat(len(population_idf))),
        'Department': list(population_idf.keys()) * len(ele_hourly),
    })
    .merge(ele_hourly[['time','Consommation']], on='time', how='left')
)

ele_dept['pop_weight'] = ele_dept['Department'].map(population_idf) / total_pop
ele_dept['Electricity_MW'] = ele_dept['Consommation'] * ele_dept['pop_weight']
ele_dept = ele_dept.drop(columns=['pop_weight'])
ele_dept


,time,Department,Consommation,Electricity_MW
0,2024-01-01,Paris,7795.895833,1282.617300
1,2024-01-01,Hauts-de-Seine,7795.895833,1041.870155
2,2024-01-01,Seine-Saint-Denis,7795.895833,1071.863210
3,2024-01-01,Val-de-Marne,7795.895833,893.137736
4,2024-01-01,Yvelines,7795.895833,932.540328
...,...,...,...,...
2923,2024-12-31,Val-de-Marne,10060.666667,1152.601476
2924,2024-12-31,Yvelines,10060.666667,1203.450840
2925,2024-12-31,Essonne,10060.666667,1085.610893
2926,2024-12-31,Val-d'Oise,10060.666667,1040.411279


In [23]:
weather_name_map = {
    'Hauts-de-Seine':'Hauts-de-Seine',
    'Seine-Saint-Denis':'Seine-Saint-Denis',
    'Val-de-Marne':'Val-de-Marne',
    'Yvelines':'Yvelines',
    'Essonne':'Essonne',
    "Val-d'Oise":"Val-d'Oise",
    'Seine-et-Marne':'Seine-et-Marne'
}
weather_long = weather_all.copy()
weather_long['Department'] = weather_long['city'].map(weather_name_map)
weather_long = weather_long.dropna(subset=['Department'])

df_dep = (
    weather_long
    .merge(ele_dept[['time','Department','Electricity_MW']], on=['time','Department'], how='inner')
    .sort_values(['Department','time']).reset_index()
)
df_dep = df_dep.drop(columns=['index','snow','city'])
df_dep


,time,tavg,tmin,tmax,prcp,wdir,wspd,wpgt,pres,tsun,Department,Electricity_MW
0,2024-01-01,7.8,5.3,10.8,9.1,<NA>,19.9,46.0,1009.7,93.0,Essonne,841.227497
1,2024-01-02,10.6,8.6,11.6,12.2,<NA>,34.3,83.0,999.2,0.0,Essonne,907.468573
2,2024-01-03,10.4,9.4,12.3,6.2,<NA>,26.5,65.0,1000.2,108.0,Essonne,901.772012
3,2024-01-04,9.4,6.9,11.0,0.0,<NA>,19.4,46.0,1006.5,137.0,Essonne,925.405774
4,2024-01-05,7.5,5.4,9.8,0.2,<NA>,14.7,48.0,1004.5,253.0,Essonne,947.142181
...,...,...,...,...,...,...,...,...,...,...,...,...
2191,2024-12-27,2.9,-0.5,6.9,0.0,<NA>,4.9,18.0,1032.8,319.0,Yvelines,1119.268692
2192,2024-12-28,-0.8,-1.7,0.0,0.0,<NA>,6.9,35.0,1031.3,40.0,Yvelines,1158.125058
2193,2024-12-29,1.8,-0.3,3.6,0.0,<NA>,5.8,21.0,1032.2,0.0,Yvelines,1124.414818
2194,2024-12-30,2.0,-0.2,3.6,0.0,<NA>,7.2,22.0,1032.5,22.0,Yvelines,1189.804261


In [24]:
df_dep.isnull().sum()

time                 0
tavg                 0
tmin                 0
tmax                 0
prcp                 0
wdir              2196
wspd                 0
wpgt                 0
pres                 0
tsun                 0
Department           0
Electricity_MW       0
dtype: int64

In [25]:
df_dep.describe()

,time,tavg,tmin,tmax,prcp,wdir,wspd,wpgt,pres,tsun,Electricity_MW
count,2196,2196.0,2196.0,2196.0,2196.0,0.0,2196.0,2196.0,2196.0,2196.0,2196.000000
mean,2024-07-01 12:00:00,12.636749,9.112933,16.443761,2.282696,<NA>,12.464617,38.2898,1016.27377,190.508197,897.863397
min,2024-01-01 00:00:00,-3.1,-6.4,-2.0,0.0,<NA>,3.9,13.0,985.0,0.0,502.099984
25%,2024-04-01 00:00:00,8.5,5.6,11.2,0.0,<NA>,8.5,28.0,1010.7,36.0,734.439561
50%,2024-07-01 12:00:00,12.6,9.2,16.4,0.2,<NA>,11.4,35.0,1016.4,150.0,849.874591
75%,2024-10-01 00:00:00,17.1,13.0,21.6,2.4,<NA>,15.3,46.0,1022.6,299.0,1027.709793
max,2024-12-31 00:00:00,28.8,22.0,37.0,73.6,<NA>,39.7,91.0,1036.6,735.0,1678.538079
std,NaN,6.054874,5.389361,7.115518,5.488241,<NA>,5.230708,13.987279,9.826993,172.857613,211.705690


In [26]:
%pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [27]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = "https://fr.wikipedia.org/wiki/D%C3%A9mographie_de_l%27%C3%8Ele-de-France"
headers = {
    "User-Agent": "BunrongBAN-ENSAEParis for coursework",
    "Accept-Language": "fr-FR,fr;q=0.9"
}

res = requests.get(url, headers=headers, timeout=30)

soup = BeautifulSoup(res.text, "html.parser")
print(soup)
# table = soup.select_one("table.wikitable.centre")
# rows = []
# for tr in table.select("tr"):
#     cells = [td.get_text(strip=True).replace("\u00A0", " ") for td in tr.find_all(["th","td"])]
#     if cells:
#         rows.append(cells)

# df = pd.DataFrame(rows[1:], columns=rows[0])
# print(df.head())


<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="fr">
<head>
<meta charset="utf-8"/>
<title>Démographie de l'Île-de-France — Wikipédia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-e

In [28]:
soup.find("table", class_ = "wikitable centre")

<table border="1" cellpadding="5" cellspacing="0" class="wikitable centre">
<tbody><tr>
<th rowspan="2">Année
</th>
<th colspan="9">Population au <time class="nowrap" data-sort-value="01-01" datetime="01-01"><abbr class="abbr" title="premier">1<sup>er</sup></abbr> janvier</time>
</th></tr>
<tr>
<th><a href="/wiki/Paris" title="Paris">Paris</a>
</th>
<th><a href="/wiki/Seine-et-Marne" title="Seine-et-Marne">Seine-et-Marne</a>
</th>
<th><a href="/wiki/Yvelines" title="Yvelines">Yvelines</a>
</th>
<th><a href="/wiki/Essonne_(d%C3%A9partement)" title="Essonne (département)">Essonne</a>
</th>
<th><a href="/wiki/Hauts-de-Seine" title="Hauts-de-Seine">Hauts-de-Seine</a>
</th>
<th><a href="/wiki/Seine-Saint-Denis" title="Seine-Saint-Denis">Seine-Saint-Denis</a>
</th>
<th><a href="/wiki/Val-de-Marne" title="Val-de-Marne">Val-de-Marne</a>
</th>
<th><a href="/wiki/Val-d%27Oise" title="Val-d'Oise">Val-d'Oise</a>
</th>
<th><b>Île-de-France</b>
</th></tr>
<tr>
<td>1968
</td>
<td align="right">2 590 

In [29]:
table = soup.find_all("table")[3]
table

<table class="wikitable" style="table-layout:fixed;width:100%;text-align:center;margin-top:10px; line-height:110%;margin-bottom: 0;font-size:95%;">
<caption class="hidden">Évolution de la population  <small>[ <a href="/wiki/Module:Donn%C3%A9es/%C3%8Ele-de-France/%C3%A9volution_population" title="Module:Données/Île-de-France/évolution population">modifier</a> ]</small>, suite (2)</caption><tbody><tr>
<th scope="col" style="background-color: #BBDEFD;">1962</th>
<th scope="col" style="background-color: #BBDEFD;">1968</th>
<th scope="col" style="background-color: #BBDEFD;">1975</th>
<th scope="col" style="background-color: #BBDEFD;">1982</th>
<th scope="col" style="background-color: #BBDEFD;">1990</th>
<th scope="col" style="background-color: #BBDEFD;">1999</th>
<th scope="col" style="background-color: #BBDEFD;">2006</th>
<th scope="col" style="background-color: #BBDEFD;">2011</th>
<th scope="col" style="background-color: #BBDEFD;">2016</th>
</tr>
<tr>
<td>8 470 015</td><td>9 248 631</td><

In [30]:
word_titles = soup.find_all('th')
word_titles

[<th scope="col" style="padding:3px;"><b>Aires</b>
 </th>,
 <th scope="col" style="padding:3px;"><b>Population</b>
 </th>,
 <th scope="col" style="padding:3px;"><b>Surface</b><br/><small>(<abbr class="abbr" title="kilomètre carré">km<sup>2</sup></abbr>)</small>
 </th>,
 <th scope="col" style="padding:3px;"><b>Densité</b><br/><small>(<abbr class="abbr" title="habitant par kilomètre carré">hab./km<sup>2</sup></abbr>)</small>
 </th>,
 <th scope="col" style="padding:5px;white-space:nowrap;"><b>2015-2021</b><br/><small>Variation <abbr class="abbr" title="population">pop.</abbr><br/>(%)</small>
 </th>,
 <th scope="row"><b>Commune de <a href="/wiki/Paris" title="Paris">Paris</a></b><sup class="reference" id="cite_ref-5"><a href="#cite_note-5"><span class="cite-bracket">[</span>5<span class="cite-bracket">]</span></a></sup><br/><small>(<a href="/wiki/D%C3%A9partement_fran%C3%A7ais" title="Département français"><abbr class="abbr" title="département">Dpt</abbr></a> <b>75</b>)</small></th>,
 <th 

In [31]:
word_table = [words.text for words in word_titles]
word_table

['Aires\n',
 'Population\n',
 'Surface(km2)\n',
 'Densité(hab./km2)\n',
 '2015-2021Variation pop.(%)\n',
 'Commune de Paris[5](Dpt 75)',
 '2\xa0133\xa0111',
 '105,4',
 '20\xa0238,2',
 '−0,6\xa0%\n',
 'Petite couronne[6](Dpts 92, 93, 94)',
 '4\xa0719\xa0328',
 '656,8',
 '7\xa0185,3',
 '+3,4\xa0%\n',
 'Grande couronne[7](Dpts 77, 78, 91, 95)',
 '5\xa0464\xa0840',
 '11\xa0250',
 '485,8',
 '+2,9\xa0%\n',
 ' Total(Île-de-France[8])',
 '12\xa0317\xa0279',
 '12\xa0012,3',
 '1\xa0025,4',
 '+0,3\xa0%\n',
 'Unité urbaineUnité urbaine de Paris\xa0: Agglomération parisienne[9]',
 '10\xa0890\xa0751',
 '2\xa0846,2',
 '3\xa0856,2',
 '+0,3\xa0%\n',
 "Aire d'attractionAire d'attraction de Paris[10]",
 '13\xa0171\xa0056',
 '18\xa0940,7',
 '695,4',
 '+0,3\xa0%\n',
 '1801',
 '1851',
 '1861',
 '1872',
 '1876',
 '1881',
 '1886',
 '1891',
 '1896',
 '1901',
 '1906',
 '1911',
 '1921',
 '1926',
 '1931',
 '1936',
 '1946',
 '1954',
 '1962',
 '1968',
 '1975',
 '1982',
 '1990',
 '1999',
 '2006',
 '2011',
 '2016',
 

In [32]:
from bs4 import BeautifulSoup
import requests, pandas as pd, re

url = "https://fr.wikipedia.org/wiki/D%C3%A9mographie_de_l%27%C3%8Ele-de-France"
headers = {
    "User-Agent": "BunrongBAN-ENSAE/1.0",
    "Accept-Language": "fr-FR,fr;q=0.9",
}
soup = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

# Find the heading "Évolution récente par département" then the next wikitable
anchor = soup.find(id="Évolution_récente_par_département")
table = anchor.find_next("table", class_="wikitable")

rows = table.find_all("tr")
word_title = [words.text for words in rows]


In [33]:
%pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 52.1 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [38]:
import requests

url = "https://fr.wikipedia.org/wiki/D%C3%A9mographie_de_l%27%C3%8Ele-de-France"
headers = {"User-Agent": "ENSAE-coursework", "Accept-Language": "fr"}
html = requests.get(url, headers=headers, timeout=30).text
df = pd.read_html(html, attrs={"class": "wikitable centre"}, header=1)[0]
df.columns = [c.split('[')[0].strip() for c in df.columns]

df.tail(10)

,Année,Paris,Seine-et-Marne,Yvelines,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d'Oise,Île-de-France
27,2014,2 220 445,1 377 846,1 421 670,1 268 228,1 597 770,1 571 028,1 365 039,1 205 539,12 027 565
28,2015,2 206 488,1 390 121,1 427 291,1 276 233,1 601 569,1 592 663,1 372 389,1 215 390,12 082 144
29,2016,2 190 327,1 397 665,1 431 808,1 287 330,1 603 268,1 606 660,1 378 151,1 221 923,12 117 132
30,2017,2 187 526,1 403 997,1 438 266,1 296 130,1 609 306,1 623 111,1 387 926,1 228 618,12 174 880
31,2018,2 175 601,1 412 516,1 441 398,1 296 641,1 619 120,1 632 677,1 396 913,1 238 581,12 213 447
32,2019,2 165 423,1 421 197,1 448 207,1 301 659,1 624 357,1 644 903,1 407 124,1 249 674,12 262 544
33,2020,2 145 906,1 428 636,1 449 723,1 306 118,1 626 213,1 655 422,1 407 972,1 251 804,12 271 794
34,2021,2 133 111,1 438 100,1 456 365,1 313 768,1 635 291,1 668 670,1 415 367,1 256 607,12 317 279
35,2022,2 113 705,1 452 399,1 470 778,1 324 546,1 647 435,1 681 725,1 419 531,1 270 845,12 380 964
36,Année,Paris,Seine-et-Marne,Yvelines,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d'Oise,Île-de-France
